# MESTRADO - DOUGLAS DE ANDRADE DE ALMEIDA

## TÍTULO: ALGORITMO DE PREDIÇÃO DE DESENVOLVIMENTO DE RESISTÊNCIA DO VÍRUS DA HEPATITE C (HCV) AOS ANTIVIRAIS DE AÇÃO DIRETA

## FEVEREIRO/2018

#### Adding paths

In [18]:
import sys
sys.path.append('../')
sys.path.append('../Scripts/')

### Libraries

In [1]:
import collections
import json
from Global_Variables import *
import Aux_Functions as auxfunctions
import numpy as np

### Open fasta file & Acession Number e Sequence

In [2]:
All_database = {}

In [3]:
Datasets_Reference = [("NS5A_1b_06.03.2018.aln","1b"),
                     ("NS5A_GEN1a_02.05.16","1a"),
                     ("NS5A_GEN3_05.05.16","3")]

In [ ]:
for dataset in Datasets_Reference:
    name_dataset = dataset[0]
    genotype_dataset = dataset[1]
    All_database[genotype_dataset] = []
    fasta = open("../Data/" + name_dataset) 
    """The file must be in the same folder as the code"""
    Sequences = collections.OrderedDict() 
    """[Acession Number, Sequence]"""
    Data_Seq = []
    """[Sequence Name,[Position,codon,AminoAcid,Resistance(True or False)]]"""

    for linha in fasta:
        linha = linha.replace("\n","")
        if linha.startswith(">"):
            name = linha[1:]
            name = name.split()
            name = name[0]
            sequence = ""
            Data_Seq.append([name])
            """Acession Number"""
        else:
            sequence += linha
            """Sequence"""
        Sequences[name] = sequence
        
    All_database[genotype_dataset].append(Sequences)

    """ Position, codon, AminoAcid e Resistance """
    for Sequence in Data_Seq:
        codons = []
        for ras in Resistance:
            """ras = resistance associated substitutions"""
            resistant = False
            Complete_sequence = Sequences[Sequence[0]]
            """Position"""
            codon = Complete_sequence[(ras*3)-3:(ras*3)]
            """Codon"""
            AminoAcid = codontable[codon] 
            """AminoAcid"""
            if AminoAcid in Resistance[ras]:
                """Resistance Check"""
                resistant = True
            codons.append([ras,codon,AminoAcid,resistant])
        Sequence.append(codons)
    
    All_database[genotype_dataset].append(Data_Seq)
    fasta.close() 

    """ Frequency Analysis """

    Frequencies = collections.OrderedDict()
    Info_Frequency = []
    for Sequence in Data_Seq:
        for data in Sequence[1]:
            """ data = [Position, codon, AminoAcid, Resistance] """
            if "Position %d" % data[0] not in Frequencies:
                Frequencies["Position %d" % data[0]] = collections.OrderedDict()
            if data[1] not in Frequencies["Position %d" % data[0]]:
                Frequencies["Position %d" % data[0]][data[1]] = 0
            Frequencies["Position %d" % data[0]] [data[1]] += 1

    for x,y in Frequencies.items():
        """Posição"""
        for key,value in y.items():
            relative_frequency = (value/len(Data_Seq))
            Info_Frequency.append([x,key,value, relative_frequency])
    All_database[genotype_dataset].append(Info_Frequency)
    frequency_file = open("../Outputs/Frequência_Genótipo %s" % genotype_dataset, "w")
    print("Genótipo %s " % genotype_dataset)
    #frequencia_file.write(("Genótipo %s " % genotype_dataset) + "\n") 
    for z in Info_Frequency:
        frequency_instance = str(z)
        frequency_instance = frequency_instance.replace("\'","")
        frequency_file.write(frequency_instance[1:-1] + "\n")
    print(np.array(Info_Frequency))
    frequency_file.close()

In [3]:
positions_list = (28,29,30,31,32,54,92,93) 
dataset = "NS5A_1b_06.03.2018.aln"